#`mountUmount(`<font size="3px" color="#01c968">`Gdrive`</font>`)`



In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive!
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

#`Setup And Update ComfyUI`



In [ ]:
from pathlib import Path

OPTIONS = {}

DRIVE_PATH = "/content/drive/MyDrive"  # @param {type:"string"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = '/content/ComfyUI'
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if DRIVE_PATH:

    WORKSPACE = DRIVE_PATH+"/ComfyUI"
    %cd {DRIVE_PATH}

![ ! -d WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

%cd {WORKSPACE}

# `Models Download`

## Download MODELS

In [ ]:
%cd {WORKSPACE}
# Install Aria2
!apt-get -y install -qq aria2


from google.colab import userdata

CIVITAI_API_TOKEN = userdata.get('CIVITAI_API_TOKEN')
print("Loaded API key:", "✅" if CIVITAI_API_TOKEN else "❌ Not found")

# Install custom nodes

def install_custom_node(url):
  %cd /content/ComfyUI/custom_nodes
  !git clone {url}

def downloadModel(url, filename = None):
  if 'huggingface.co' in url:
    if filename is None:
      filename = url.split('/')[-1]
      filename = filename.removesuffix('?download=true')
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    if filename:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}?token={CIVITAI_API_TOKEN} -o {filename}
    else:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}?token={CIVITAI_API_TOKEN}


# install custom nodes
#install_custom_node('https://github.com/ltdrdata/ComfyUI-Manager.git')
#install_custom_node('https://github.com/ltdrdata/ComfyUI-Impact-Pack')


# download models
%cd ./models/checkpoints

# CyberRealistic Pony
downloadModel('https://civitai.com/api/download/models/2071650')

# Protovision XL
#downloadModel('https://civitai.com/api/download/models/201514')
# Juggernaut XL Hyper
#downloadModel('https://civitai.com/api/download/models/471120')
# Juppernaut X
#downloadModel('https://civitai.com/api/download/models/456194')
# Juggernaut v8
#downloadModel('https://civitai.com/api/download/models/288982')

# Huggingface example
#downloadModel('https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors?download=true')

%cd {WORKSPACE}

## OTHERS

In [ ]:
# SD1.5
!wget -c https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -P ./models/checkpoints/

# Some SD1.5 anime style
!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors -P ./models/checkpoints/

# VAE
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/

## LIST MODELS

In [ ]:
!ls -al ./models/checkpoints/

# `START ComfyUI  & Expose Server (MANUAL)`

## Download Prerequisits

In [6]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

--2026-02-02 09:30:55--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2026.1.2/cloudflared-linux-amd64.deb [following]
--2026-02-02 09:30:56--  https://github.com/cloudflare/cloudflared/releases/download/2026.1.2/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/ca5c1d34-b657-49b6-8531-2e60b48ba318?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-02-02T10%3A09%3A26Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&

In [ ]:
!python main.py --dont-print-server

Checkpoint files will always be loaded safely.
Found comfy_kitchen backend cuda: {'available': True, 'disabled': True, 'unavailable_reason': None, 'capabilities': ['apply_rope', 'apply_rope1', 'dequantize_nvfp4', 'dequantize_per_tensor_fp8', 'quantize_nvfp4', 'quantize_per_tensor_fp8', 'scaled_mm_nvfp4']}
Found comfy_kitchen backend eager: {'available': True, 'disabled': False, 'unavailable_reason': None, 'capabilities': ['apply_rope', 'apply_rope1', 'dequantize_nvfp4', 'dequantize_per_tensor_fp8', 'quantize_nvfp4', 'quantize_per_tensor_fp8', 'scaled_mm_nvfp4']}
Found comfy_kitchen backend triton: {'available': True, 'disabled': True, 'unavailable_reason': None, 'capabilities': ['apply_rope', 'apply_rope1', 'dequantize_nvfp4', 'dequantize_per_tensor_fp8', 'quantize_nvfp4', 'quantize_per_tensor_fp8']}
Total VRAM 15095 MB, total RAM 12976 MB
pytorch version: 2.10.0+cu128
xformers version: 0.0.34
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync
Using async weight o

# `START ComfyUI & Expose Server`

## Download Prerequisits

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

## CF Tunnel

In [ ]:
import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

## localtunnel

In [ ]:
# localtunnel
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()
%cd /content/ComfyUI
!python main.py --dont-print-server